In [4]:
# rt (= 연구논문 추정) 목록 수집 후 CSV 저장 (한/영 전체 + keyword1/2/3 포함)
import requests, json
import pandas as pd
from datetime import datetime

try:
    base = "https://library.kimm.re.kr/openAPI/openAPI_allsearch.do"

    # lang 제거하여 한국어/영어 전체 결과 수집
    params = {
        "search_gubun": "rt",  # 연구논문
        "pageSize": 30000,
        "page": 1,
        # "lang": "kor",  # 필터 제거
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://library.kimm.re.kr/",
    }

    sess = requests.Session()
    sess.headers.update(headers)

    def to_int(val, default=None):
        try:
            if val is None or val == "":
                return default
            return int(float(str(val)))
        except Exception:
            return default

    def extract_keyword(it, keys):
        for k in keys:
            v = it.get(k)
            if v:
                return str(v).strip()
        return ""

    def map_item(it):
        # keyword1/2/3는 다양한 키 변형이 있을 수 있어 몇 가지를 병행 체크
        kw1 = extract_keyword(it, ["keyword1", "keyworkd1", "keyword_1"])  # keyworkd* 오타 대비
        kw2 = extract_keyword(it, ["keyword2", "keyworkd2", "keyword_2"]) 
        kw3 = extract_keyword(it, ["keyword3", "keyworkd3", "keyword_3"]) 
        keywords_joined = "; ".join([k for k in [kw1, kw2, kw3] if k])

        return {
            "title": (it.get("title") or "").strip(),
            "author": (it.get("author") or "").strip(),
            "publisher": it.get("publisher", ""),
            "pubyear": it.get("pubyear", ""),
            "isbn": it.get("isbn", ""),
            "issn": it.get("issn", ""),
            "location": it.get("location", ""),
            "bibctrlno": it.get("bibctrlno", ""),
            "detail_url": it.get("libURL", ""),
            "type": "연구논문",
            "keyword1": kw1,
            "keyword2": kw2,
            "keyword3": kw3,
            "keywords": keywords_joined,
        }

    def fetch_page(page: int):
        q = dict(params)
        q["page"] = page
        print("[RUN] GET", base, q)
        resp = sess.get(base, params=q, timeout=60)
        print("[INFO] status:", resp.status_code, resp.headers.get("content-type"))
        resp.raise_for_status()

        js = resp.json()
        xmlData = js.get("xmlData") if isinstance(js, dict) else None

        page_rows = []
        pageing = {}
        if xmlData and isinstance(xmlData, dict):
            lst = xmlData.get("list") or []
            if lst and isinstance(lst, list):
                block = lst[0]
                row_list = block.get("rowList") or []

                # 외부 crawler 파서가 있으면 사용 (keyword 포함 여부는 해당 파서 정의에 따름)
                if "crawler" in globals():
                    try:
                        tmp = crawler.parse_json_results(row_list, default_type="연구논문")
                        # keyword1/2/3 컬럼이 없다면 최소한 매핑해서 보강
                        for it, base_row in zip(row_list, tmp):
                            if "keyword1" not in base_row:
                                base_row["keyword1"] = extract_keyword(it, ["keyword1", "keyworkd1", "keyword_1"]) 
                            if "keyword2" not in base_row:
                                base_row["keyword2"] = extract_keyword(it, ["keyword2", "keyworkd2", "keyword_2"]) 
                            if "keyword3" not in base_row:
                                base_row["keyword3"] = extract_keyword(it, ["keyword3", "keyworkd3", "keyword_3"]) 
                            ks = [base_row.get("keyword1", ""), base_row.get("keyword2", ""), base_row.get("keyword3", "")]
                            base_row["keywords"] = "; ".join([k for k in ks if k])
                        page_rows = tmp
                    except Exception:
                        # 파서 실패 시 로컬 매퍼로 대체
                        page_rows = [map_item(it) for it in row_list]
                else:
                    page_rows = [map_item(it) for it in row_list]

                pageing = block.get("pageing", {})
        else:
            print("[WARN] 예상과 다른 응답 구조, 텍스트 미리보기:")
            print(resp.text[:500])

        # 다양한 표기(totalPage/totalpage/total_page) 대응 및 안전 변환
        total_raw = pageing.get("total") if isinstance(pageing, dict) else None
        totalPage_raw = None
        if isinstance(pageing, dict):
            for k in ("totalPage", "totalpage", "total_page"):
                if k in pageing:
                    totalPage_raw = pageing.get(k)
                    break
        return page_rows, total_raw, totalPage_raw

    # 첫 페이지 조회로 전체 페이지 파악
    all_rows = []
    seen_ids = set()

    first_rows, total, totalPage = fetch_page(1)
    for r in first_rows:
        key = r.get("bibctrlno") or (r.get("title"), r.get("author"))
        if key not in seen_ids:
            seen_ids.add(key)
            all_rows.append(r)

    tp = to_int(totalPage, 1) or 1
    print(f"[INFO] page=1 size={len(first_rows)} collected={len(all_rows)} total={total} totalPage_raw={totalPage} totalPage_int={tp}")

    # 남은 페이지 수집 (안전 장치)
    MAX_PAGES = 2000
    for p in range(2, min(tp, MAX_PAGES) + 1):
        page_rows, total, totalPage = fetch_page(p)
        for r in page_rows:
            key = r.get("bibctrlno") or (r.get("title"), r.get("author"))
            if key not in seen_ids:
                seen_ids.add(key)
                all_rows.append(r)
        print(f"[INFO] page={p} size={len(page_rows)} collected={len(all_rows)}")

    if not all_rows:
        print("[WARN] 수집된 행이 없습니다. 파라미터나 권한을 확인하세요.")

    # DataFrame 생성 (전체)
    df_rt = pd.DataFrame(all_rows)
    print("[INFO] DataFrame(all):", df_rt.shape)

    if not df_rt.empty:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_name = f"KIMM_rt_{ts}.csv"
        # Excel 호환을 위해 UTF-8 BOM
        df_rt.to_csv(csv_name, index=False, encoding="utf-8-sig")
        print("[DONE] CSV 저장:", csv_name)
        # 상위 3개 미리보기
        try:
            display(df_rt.head(3))
        except Exception:
            print(df_rt.head(3))
    else:
        print("[INFO] 저장할 데이터 없음")

except Exception as e:
    print("[ERROR] 실행 실패:", e)

[RUN] GET https://library.kimm.re.kr/openAPI/openAPI_allsearch.do {'search_gubun': 'rt', 'pageSize': 30000, 'page': 1}
[INFO] status: 200 application/json;charset=UTF-8
[INFO] page=1 size=27250 collected=27250 total=27250 totalPage_raw=1.0 totalPage_int=1
[INFO] status: 200 application/json;charset=UTF-8
[INFO] page=1 size=27250 collected=27250 total=27250 totalPage_raw=1.0 totalPage_int=1
[INFO] DataFrame(all): (27250, 14)
[INFO] DataFrame(all): (27250, 14)
[DONE] CSV 저장: KIMM_rt_20250829_174105.csv
[DONE] CSV 저장: KIMM_rt_20250829_174105.csv


,title,author,publisher,pubyear,isbn,issn,location,bibctrlno,detail_url,type,keyword1,keyword2,keyword3,keywords
0,,신영관;최원석;김훈영;조광우;조성학,"KoNTRS(나노기술연구협의회), NTRA(나노융합산업연구조합);KoNTRS(나노기...",2019.07.05,,,[],108468,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,
1,[001] 및 [011] 방향 분극의 압전 단결정 PMN-PZT를 이용한 진동 에너...,선경호;Kyung Ho Sun;김영철;Young-Cheol Kim;김재은;Jae E...,한국소음진동공학회;한국소음진동공학회,2014.10.30,,,[],97180,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,
2,[001] 및 [011] 방향 분극의 압전 단결정 PMN-PZT를 이용한\n진동 에...,김재은;Jae Eun Kim;김영철;Young-Cheol Kim;선경호;Kyung ...,한국소음진동공학회;한국소음진동공학회,2014.11.24,,,[],97458,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,


In [5]:
# 2023~2025 연도 필터링 및 별도 CSV 저장
import re
import pandas as pd
from datetime import datetime

# 전 셀에서 df_rt가 생성되어 있어야 합니다.
try:
    if 'df_rt' not in globals() or df_rt is None or df_rt.empty:
        raise RuntimeError('df_rt가 비어있거나 없습니다. 먼저 1번 셀을 실행하세요.')

    def to_year(s):
        try:
            m = re.search(r"(19|20)\d{2}", str(s))
            return int(m.group(0)) if m else None
        except Exception:
            return None

    df = df_rt.copy()
    df['year'] = df['pubyear'].apply(to_year)
    df_filtered = df[(df['year'] >= 2023) & (df['year'] <= 2025)].copy()
    print('[INFO] DataFrame(filtered 2023~2025):', df_filtered.shape)

    if not df_filtered.empty:
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        csv_name = f'KIMM_rt_{ts}_y2023_2025.csv'
        df_filtered.to_csv(csv_name, index=False, encoding='utf-8-sig')
        print('[DONE] CSV 저장:', csv_name)
        try:
            display(df_filtered.head(3))
        except Exception:
            print(df_filtered.head(3))
    else:
        print('[INFO] 저장할 데이터 없음 (필터 결과 0건)')

except Exception as e:
    print('[ERROR] 실행 실패:', e)

[INFO] DataFrame(filtered 2023~2025): (1446, 15)
[DONE] CSV 저장: KIMM_rt_20250829_174129_y2023_2025.csv


,title,author,publisher,pubyear,isbn,issn,location,bibctrlno,detail_url,type,keyword1,keyword2,keyword3,keywords,year
13,100-kWe-class Supercritical Organic Rankine Cy...,임형수;최범석;박무룡;황순찬;박준영;서정민;방제성;김수원;임영철;박세진;정희찬;이동...,대한기계학회;대한기계학회,2024.12.01,,,[],121102,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,,2024.0
100,"2,000℃에서 사용 가능한 효율 40%의 열기관",박중호,대한기계학회(KSME);대한기계학회(KSME),2023.08.01,,,[],120213,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,,2023.0
132,2022년 압축기 분야 연구동향,서정민,한국유체기계학회(KSFM);한국유체기계학회(KSFM),2023.04.01,,,[],114443,http://https://library.kimm.re.kr/search/catal...,연구논문,,,,,2023.0
